# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

PATH = "/Users/admin/edu/NLP/"

with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [2]:
tree = trees[0]
for node in tree:
    head = node["head"]
    print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

У <-- домі
домі <-- була
римського <-- патриція
патриція <-- домі
Руфіна <-- патриція
була <-- root
прегарна <-- фреска
фреска <-- була
, <-- зображення
зображення <-- фреска
Венери <-- зображення
та <-- Адоніса
Адоніса <-- Венери
. <-- була


## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [3]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

def oracle(top_stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         top_queue["head"] < top_stack["id"]:
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [4]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack[-1] if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"] for i in stack])
            print("Queue:", [i["form"] for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT']
Queue: ['У', 'домі', 'римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: []
Actions.SHIFT
Stack: ['ROOT', 'У']
Queue: ['домі', 'римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: []
Actions.LEFT
Stack: ['ROOT']
Queue: ['домі', 'римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT', 'домі']
Queue: ['римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT', 'домі', 'римського']
Queue: ['патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: [(1, 2)]
Actions.LEFT
Stack: ['ROOT', 'домі']
Queue: ['патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зобр

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [5]:
def extract_features(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
        if stack_top["feats"]:
            for k, v in stack_top["feats"].items():
                features["s0-" + k] = v
    if len(stack) > 1:
        features["s1-tag"] = stack_top["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        if queue_top["feats"]:
            for k, v in queue_top["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    if stack and queue:
        features["distance"] = queue[0]["id"] - stack[-1]["id"]
    return features

## Prepare train and test data

In [6]:
def get_data(trees):
    features, labels = [], []
    for tree in trees:
        stack, queue, relations = [ROOT], tree[:], []

        while queue or stack:
            action = oracle(stack[-1] if len(stack) > 0 else None,
                            queue[0] if len(queue) > 0 else None,
                            relations)
            features.append(extract_features(stack, queue))
            labels.append(action.value)
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return features, labels

In [7]:
features, labels = get_data([tree])
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features:")
for word in features:
    print(word)
    break

Number of words: 14
Number of actions: 29
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'У', 'q0-lemma': 'у', 'q0-tag': 'ADP', 'q0-Case': 'Loc', 'q1-word': 'домі', 'q1-tag': 'NOUN', 'q2-tag': 'ADJ', 'q3-tag': 'NOUN', 'distance': 1}


In [8]:
train_features, train_labels = get_data(trees)

print(len(train_features), len(train_labels))

154709 154709


In [9]:
train_df = pd.DataFrame(train_features)

In [10]:
train_df[[x for x in train_df.columns if 'tag' in x]].head()

,q0-tag,q1-tag,q2-tag,q3-tag,s0-tag,s1-tag
0,ADP,NOUN,ADJ,NOUN,ROOT,NaN
1,NOUN,ADJ,NOUN,PROPN,ADP,ADP
2,NOUN,ADJ,NOUN,PROPN,ROOT,NaN
3,ADJ,NOUN,PROPN,VERB,NOUN,NOUN
4,NOUN,PROPN,VERB,ADJ,ADJ,ADJ


In [11]:
with open(PATH + "/uk_iu-ud-test.conllu", "r") as f:
    data = f.read()

test_trees = parse(data)
test_features, test_labels = get_data(test_trees)
print(len(test_features), len(test_labels))

30661 30661


## Train a classifier

In [12]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features + test_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  111525


In [13]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

154709 30661


In [14]:
lrc = LogisticRegression(random_state=42)
lrc.fit(train_features_vectorized, train_labels)
predicted = lrc.predict(test_features_vectorized)

In [15]:
print(classification_report(test_labels, predicted))

             precision    recall  f1-score   support

       left       0.84      0.86      0.85      7352
     reduce       0.81      0.73      0.77      8370
      right       0.72      0.78      0.75      7182
      shift       0.86      0.86      0.86      7757

avg / total       0.81      0.81      0.81     30661



In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_fscore_support, f1_score, roc_auc_score, roc_curve, auc

classes = ['left', 'reduce', 'right', 'shift']
conf_matrix = confusion_matrix(test_labels, predicted, labels=classes)
pd.DataFrame(conf_matrix, columns=classes, index=classes)
# conf_matrix_lemma

,left,reduce,right,shift
left,6296,394,326,336
reduce,557,6103,1372,338
right,212,880,5630,460
shift,392,184,494,6687


## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [34]:
def dep_parse(sentence, oracle, vectorizer):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [35]:
print([node["form"] for node in test_trees[0]])
print(dep_parse(test_trees[0], lrc, vec))
print([(node["id"], node["head"]) for node in test_trees[0]])

['Зречення', 'культурної', 'ідентичності', '—', 'це', 'втрата', 'свободи', 'й', 'самовладності', '.']
[(1, 0), (2, 3), (3, 1), (4, 6), (5, 6), (6, 1), (7, 6), (8, 9), (9, 7), (10, 0)]
[(1, 6), (2, 3), (3, 1), (4, 6), (5, 6), (6, 0), (7, 6), (8, 9), (9, 7), (10, 6)]


In [36]:
total, tp = 0, 0
for tree in test_trees:
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))

Total: 14939
Correctly defined: 10236
UAS: 0.69


## New Features

I decided to add few more features to the model:
<li>position in the sentance for q0 and s0</li>
<li>is the words q0 and s0 are the Beggining or End of the sentence</li>
<li>is the words q0 and s0 are neibhors in the sentence</li>
<li>is punctuation before or after q0 (but not the '.')</li>

New feature generation function and normalize few features:
<li>position and distance in the sentence. I assume, that the mean length of the sentence is 6, and the upper border is 12, so I am going to divide position in the sentence in 12, and if it > 1 the The value is 1</li>

In [33]:
def extract_features_upd(stack, queue, sent_len):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
        features["s0-start"] = stack_top["id"] == 0
        features["s0-eos"] = sent_len - stack_top["id"] == 2
        features["s0-posit"] = 1.0 if stack_top["id"] > 12 else (stack_top["id"] + 1 / 12.0)
        if stack_top["feats"]:
            for k, v in stack_top["feats"].items():
                features["s0-" + k] = v
    if len(stack) > 1:
        features["s1-tag"] = stack_top["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        features["q0-start"] = queue_top["id"] == 0
        features["q0-eos"] = sent_len - queue_top["id"] == 2
        features["q0-posit"] = 1.0 if queue_top["id"] > 12 else (queue_top["id"] + 1 / 12.0)
        if queue_top["feats"]:
            for k, v in queue_top["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        features["q0-bef_punct"] = queue_next["upostag"] == "PUNCT" and queue_next["form"] != "."
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    if stack and queue:
        features["distance"] = (queue[0]["id"] - stack[-1]["id"]) / 12.0
        features["is-neib"] = abs(queue[0]["id"] - stack[-1]["id"]) == 1
    return features

New dataframe genearator

In [34]:
def get_data_upd(trees):
    features, labels = [], []
    for tree in trees:
        stack, queue, relations = [ROOT], tree[:], []

        while queue or stack:
            action = oracle(stack[-1] if len(stack) > 0 else None,
                            queue[0] if len(queue) > 0 else None,
                            relations)
            features.append(extract_features_upd(stack, queue, len(tree)))
            labels.append(action.value)
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return features, labels

In [35]:
train_features_upd, train_labels_upd = get_data_upd(trees)
print(len(train_features), len(train_labels))

154709 154709


In [36]:
with open(PATH + "/uk_iu-ud-test.conllu", "r") as f:
    data_test = f.read()

test_trees = parse(data_test)
test_features_upd, test_labels_upd = get_data_upd(test_trees)
print(len(test_features_upd), len(test_labels_upd))

30661 30661


In [37]:
vectorizer = DictVectorizer()
vec_upd = vectorizer.fit(train_features_upd + test_features_upd)

print("\nTotal number of features: ", len(vec_upd.get_feature_names()))


Total number of features:  111533


In [38]:
train_features_upd_vectorized = vec_upd.transform(train_features_upd)
test_features_upd_vectorized = vec_upd.transform(test_features_upd)

print(len(train_features_upd_vectorized.toarray()), len(test_features_upd_vectorized.toarray()))

154709 30661


In [39]:
lru = LogisticRegression(random_state=42)
lru.fit(train_features_upd_vectorized, train_labels_upd)
predicted_upd = lru.predict(test_features_upd_vectorized)

In [40]:
print(classification_report(test_labels_upd, predicted_upd))

             precision    recall  f1-score   support

       left       0.85      0.86      0.86      7352
     reduce       0.81      0.74      0.77      8370
      right       0.72      0.79      0.75      7182
      shift       0.86      0.86      0.86      7757

avg / total       0.81      0.81      0.81     30661



We can see, that after adding few new features the scores very slightly increased, so we can notice that this new features are not important and almoast useless.

## Different models

1. Logistic regression with cross-validation

In [41]:
from sklearn.linear_model import LogisticRegressionCV

lrcv = LogisticRegressionCV(random_state=42, multi_class='multinomial')
lrcv.fit(train_features_upd_vectorized, train_labels_upd)
predicted_lrcv = lrcv.predict(test_features_upd_vectorized)

In [42]:
print(classification_report(test_labels_upd, predicted_lrcv))

             precision    recall  f1-score   support

       left       0.85      0.87      0.86      7352
     reduce       0.83      0.73      0.78      8370
      right       0.73      0.81      0.77      7182
      shift       0.88      0.88      0.88      7757

avg / total       0.82      0.82      0.82     30661



Here you can see a slight improvement in the results.

2. SVM with cross-validation

In [ ]:
from sklearn.svm import SVC

clf_svm = SVC(cache_size=1000)
clf_svm.fit(train_features_upd_vectorized, train_labels_upd)

In [27]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'accuracy']
c_params = {'C': [1, 10, 100]}

clf_svm_temp = GridSearchCV(SVC(cache_size=1000), c_params, cv=3, return_train_score=False, n_jobs=4,
                            scoring=scoring, refit=False)
clf_svm_temp.fit(train_features_upd_vectorized, train_labels_upd)

KeyboardInterrupt: 